In [1]:
!pip install boto3
!pip install configparser

#AWS
import boto3
import json
import os

#PDFTEXT
import urllib.request
import pdftotext

import re
import pprint
from textblob import TextBlob

## 1. Import pdf text from /Download folder

### In the folder Download you have a lot of pdf from ArXiv.com. 
### You can also import your own pdf in the folder "Teacher" and use the code below to extract the entities names

In [2]:
path='./Download'
first_article=os.listdir(path)[0]
last_article=os.listdir(path)[-1]

In [3]:
print(first_article)
print(last_article)

Assouad type dimensions of parabolic Julia sets.pdf
Temporal Difference Learning for Model Predictive Control.pdf


In [4]:
# Load your PDF
def load_pdf(article):
    try:
        with open(f'Download/{article}', "rb") as f:
            pdf = pdftotext.PDF(f)
            mypdftext="\n\n".join(pdf)
    except:
        with open(f'Teacher/{article}', "rb") as f:
            pdf = pdftotext.PDF(f)
            mypdftext="\n\n".join(pdf)
    return mypdftext

In [5]:
pdf=load_pdf(first_article)

In [6]:
#First filter to have only the References from a pdf
def after_references(mypdftext=load_pdf(first_article)): 
    keyword1 = 'References'
    keyword2 = 'REFERENCES'
    keyword3 = 'R EFERENCES'
    keyword4 = 'Reference'
    keyword5='[1]' 

    if keyword1 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword1)
    elif keyword2 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword2)
    elif keyword3 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword3)
    elif keyword4 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword4)
    elif keyword5 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword5)
    else:
        after_keyword = mypdftext[:10000]
    return after_keyword

In [7]:
#Display references in the pdf
references=after_references(pdf)
references

'\r\n[1] Aaronson, J., Denker, M., and Urbański, M. (1993). Ergodic theory for Markov fibred systems\r\nand parabolic rational maps. Trans. Amer. Math. Soc., 337(2):495–548.\r\n[2] Anderson, T. C., Hughes, K., Roos, J., and Seeger, A. (2021). Lp → Lq bounds for spherical\r\nmaximal operators. Math. Z., 297(3-4):1057–1074.\r\n[3] Beardon, A. F. (1991). Iteration of rational maps. Springer Verlag, New York.\r\n[4] Bishop, C. J. and Peres, Y. (2017). Fractals in Probability and Analysis, volume 162 of Cambridge\r\nStudies in Advanced Mathematics. Cambridge.\r\n[5] Bowen, R. (1979). Hausdorff dimension of quasi-circles. Pub. Math. IHES, 50:11–25.\r\n[6] Cheraghi, D., DeZotti, A., and Yang, F. (2020). Dimension paradox of irrationally indifferent\r\nattractors. preprint, available at: https://arxiv.org/abs/2003.12340.\r\n[7] Denker, M. and Urbański, M. (1991a). Absolutely continuous invariant measures for expansive\r\nrational maps with rationally indifferent periodic points. Forum Math.,

In [8]:
#First cleaning of this text

replacement_patterns = [
    (r'won\'t', 'will not'),
    (r'can\'t', 'cannot'),
    (r'i\'m', 'i am'),
    (r'ain\'t', 'is not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)n\'t', '\g<1> not'),
    (r'(\w+)\'ve', '\g<1> have'),
    (r'(\w+)\'s', '\g<1> is'),
    (r'(\w+)\'re', '\g<1> are'),
    (r'(\w+)\'d', '\g<1> would'),
]

class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns): 
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s) 
        return s

replacer=RegexpReplacer()

references=replacer.replace(references)
print(references)


[1] Aaronson, J., Denker, M., and Urbański, M. (1993). Ergodic theory for Markov fibred systems
and parabolic rational maps. Trans. Amer. Math. Soc., 337(2):495–548.
[2] Anderson, T. C., Hughes, K., Roos, J., and Seeger, A. (2021). Lp → Lq bounds for spherical
maximal operators. Math. Z., 297(3-4):1057–1074.
[3] Beardon, A. F. (1991). Iteration of rational maps. Springer Verlag, New York.
[4] Bishop, C. J. and Peres, Y. (2017). Fractals in Probability and Analysis, volume 162 of Cambridge
Studies in Advanced Mathematics. Cambridge.
[5] Bowen, R. (1979). Hausdorff dimension of quasi-circles. Pub. Math. IHES, 50:11–25.
[6] Cheraghi, D., DeZotti, A., and Yang, F. (2020). Dimension paradox of irrationally indifferent
attractors. preprint, available at: https://arxiv.org/abs/2003.12340.
[7] Denker, M. and Urbański, M. (1991a). Absolutely continuous invariant measures for expansive
rational maps with rationally indifferent periodic points. Forum Math., 3(6):561–580.
[8] Denker, M. and Urb

## 2. AWS Comprehend service

### If Cli doesn't works, insert your variable here and execute the cell below , otherwise skip this cell :

In [9]:
ACCESS_ID='...'
ACCESS_KEY='...'
session_token='...'

In [10]:
#With CLI
def compehend(references):
    try:
        comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')
        input_text = references
        result_json=comprehend.detect_entities(Text=input_text, LanguageCode='en')
        print(result_json)
    except :
        comprehend = boto3.client(service_name='comprehend', region_name='us-east-1',aws_access_key_id=ACCESS_ID,
                                            aws_secret_access_key= ACCESS_KEY,aws_session_token=session_token)
        input_text = references
        result_json=comprehend.detect_entities(Text=input_text, LanguageCode='en')
    return result_json

In [11]:
result_json= compehend(references)
result_json

{'Entities': [{'Score': 0.9867931604385376, 'Type': 'PERSON', 'Text': 'Aaronson, J.', 'BeginOffset': 6, 'EndOffset': 18}, {'Score': 0.9666887521743774, 'Type': 'PERSON', 'Text': 'Denker, M.', 'BeginOffset': 20, 'EndOffset': 30}, {'Score': 0.9715626239776611, 'Type': 'PERSON', 'Text': 'Urbański, M.', 'BeginOffset': 36, 'EndOffset': 49}, {'Score': 0.9946287274360657, 'Type': 'DATE', 'Text': '1993', 'BeginOffset': 51, 'EndOffset': 55}, {'Score': 0.6216310262680054, 'Type': 'TITLE', 'Text': 'Ergodic theory for Markov', 'BeginOffset': 58, 'EndOffset': 83}, {'Score': 0.5866131782531738, 'Type': 'TITLE', 'Text': 'systems\r\nand', 'BeginOffset': 91, 'EndOffset': 103}, {'Score': 0.6200372576713562, 'Type': 'OTHER', 'Text': '337', 'BeginOffset': 154, 'EndOffset': 157}, {'Score': 0.7530454397201538, 'Type': 'OTHER', 'Text': '495–548', 'BeginOffset': 161, 'EndOffset': 168}, {'Score': 0.9599947333335876, 'Type': 'PERSON', 'Text': 'Anderson, T. C.', 'BeginOffset': 175, 'EndOffset': 190}, {'Score': 

{'Entities': [{'Score': 0.9867931604385376,
   'Type': 'PERSON',
   'Text': 'Aaronson, J.',
   'BeginOffset': 6,
   'EndOffset': 18},
  {'Score': 0.9666887521743774,
   'Type': 'PERSON',
   'Text': 'Denker, M.',
   'BeginOffset': 20,
   'EndOffset': 30},
  {'Score': 0.9715626239776611,
   'Type': 'PERSON',
   'Text': 'Urbański, M.',
   'BeginOffset': 36,
   'EndOffset': 49},
  {'Score': 0.9946287274360657,
   'Type': 'DATE',
   'Text': '1993',
   'BeginOffset': 51,
   'EndOffset': 55},
  {'Score': 0.6216310262680054,
   'Type': 'TITLE',
   'Text': 'Ergodic theory for Markov',
   'BeginOffset': 58,
   'EndOffset': 83},
  {'Score': 0.5866131782531738,
   'Type': 'TITLE',
   'Text': 'systems\r\nand',
   'BeginOffset': 91,
   'EndOffset': 103},
  {'Score': 0.6200372576713562,
   'Type': 'OTHER',
   'Text': '337',
   'BeginOffset': 154,
   'EndOffset': 157},
  {'Score': 0.7530454397201538,
   'Type': 'OTHER',
   'Text': '495–548',
   'BeginOffset': 161,
   'EndOffset': 168},
  {'Score': 0.

###  Export of entities in json

In [12]:
#Creation of json in order to keep only relevant information
def json_creation(result_json):
    output_json=[]
    i=0
    for json in result_json['Entities']:
        data={}
        if json['Type']== 'PERSON' and json['Score']>0.90:
            data["index"] = i
            data["Person"] = json['Text']
            data["Fiability"] = json['Score']
            output_json.append(data)
            i += 1
    return output_json

In [13]:
import json
import os

#Convert the array to json file
def create_json_file(jsonobject, filepath):
    with open(filepath, 'w') as outfile:
        json.dump(jsonobject, outfile)

In [14]:
#Import the json in my desktop
filename = "./Download/EntitiesPDF.json"
create_json_file(json_creation(result_json), filename)

#Display the json result
f = open("./Download/EntitiesPDF.json", "r")
print(f.read())

[{"index": 0, "Person": "Aaronson, J.", "Fiability": 0.9867931604385376}, {"index": 1, "Person": "Denker, M.", "Fiability": 0.9666887521743774}, {"index": 2, "Person": "Urban\u0301ski, M.", "Fiability": 0.9715626239776611}, {"index": 3, "Person": "Anderson, T. C.", "Fiability": 0.9599947333335876}, {"index": 4, "Person": "Hughes, K.", "Fiability": 0.9457246661186218}, {"index": 5, "Person": "Roos, J.", "Fiability": 0.9628537893295288}, {"index": 6, "Person": "Seeger, A.", "Fiability": 0.9617085456848145}, {"index": 7, "Person": "Beardon, A. F.", "Fiability": 0.9783458113670349}, {"index": 8, "Person": "Bishop, C. J.", "Fiability": 0.9074485301971436}, {"index": 9, "Person": "Peres, Y.", "Fiability": 0.9677588939666748}, {"index": 10, "Person": "Bowen, R.", "Fiability": 0.983867347240448}, {"index": 11, "Person": "Cheraghi, D.", "Fiability": 0.9779359698295593}, {"index": 12, "Person": "DeZotti, A.", "Fiability": 0.9747372269630432}, {"index": 13, "Person": "Yang, F.", "Fiability": 0.91

## 3. Test the code with your pdf

In [15]:
#HERE YOUR OWN PDF:
path='./Teacher/'
article_teacher=os.listdir(path)[-1]
print(article_teacher)

Searching for the Origin of Double-peaked Broad Emission Lines in a Merging Galaxy with EVN.pdf


In [16]:
my_article=load_pdf(article_teacher)

In [17]:
#Display references in the pdf
references=after_references(my_article)
references

'\r\n[1] M. Volonteri, F. Haardt, P. Madau, The Assembly and Merging History of Supermassive Black Holes in Hierarchical Models of Galaxy Formation ApJ 582 (2003) 559\r\n[arXiv:astro-ph/0207276].\r\n[2] M. C. Begelman, R. D. Blandford, and M. J. Rees, Massive black hole binaries in active\r\ngalactic nuclei, Nature 287 (1980) 307.\r\n[3] L. Blecha, T. J. Cox, A. Loeb, and L. Hernquist Recoiling black holes in merging galaxies:\r\nrelationship to active galactic nucleus lifetimes, starbursts and the MBH-𝜎 ∗ relation, MNRAS\r\n412 (2011) 2154 [arXiv:1009.4940 ].\r\n[4] P. R. Capelo, M. Volonteri, M. Dotti, J. M. Bellovary, L. Mayer, and F. Governato Growth and\r\nactivity of black holes in galaxy mergers with varying mass ratios, MNRAS 447 (2015) 2131\r\n[arXiv:1409.0004].\r\n[5] T. An, P. Mohan, and S. Frey, VLBI Studies of DAGN and SMBHB Hosting Galaxies, Radio\r\nScience 53 (2018) 2111 [arXiv:1808.07651].\r\n[6] C. Rodriguez, G. B. Taylor, R. T. Zavala, A. B. Peck, L. K. Pollack, and 

In [18]:
#AWS comprehend
result_json=compehend(references)

{'Entities': [{'Score': 0.9895498156547546, 'Type': 'PERSON', 'Text': 'M. Volonteri', 'BeginOffset': 6, 'EndOffset': 18}, {'Score': 0.9617865681648254, 'Type': 'PERSON', 'Text': 'F. Haardt', 'BeginOffset': 20, 'EndOffset': 29}, {'Score': 0.9832359552383423, 'Type': 'PERSON', 'Text': 'P. Madau', 'BeginOffset': 31, 'EndOffset': 39}, {'Score': 0.9147467613220215, 'Type': 'TITLE', 'Text': 'The Assembly and Merging History of Supermassive Black Holes in Hierarchical Models of Galaxy Formation', 'BeginOffset': 41, 'EndOffset': 144}, {'Score': 0.9552740454673767, 'Type': 'DATE', 'Text': '2003', 'BeginOffset': 154, 'EndOffset': 158}, {'Score': 0.9905604124069214, 'Type': 'PERSON', 'Text': 'M. C. Begelman', 'BeginOffset': 196, 'EndOffset': 210}, {'Score': 0.9953071475028992, 'Type': 'PERSON', 'Text': 'R. D. Blandford', 'BeginOffset': 212, 'EndOffset': 227}, {'Score': 0.9941856861114502, 'Type': 'PERSON', 'Text': 'M. J. Rees', 'BeginOffset': 233, 'EndOffset': 243}, {'Score': 0.9515606164932251, 

In [19]:
#Creation of json in order to keep only relevant information
import json
import os

#Import the json in my desktop
filename = "./Teacher/AWS_json_teacher.json"
create_json_file(json_creation(result_json), filename)

#Display the json result
f = open("./Teacher/AWS_json_teacher", "r")
print(f.read())

[{"index": 0, "Person": "M. Volonteri", "Fiability": 0.9895498156547546}, {"index": 1, "Person": "F. Haardt", "Fiability": 0.9617865681648254}, {"index": 2, "Person": "P. Madau", "Fiability": 0.9832359552383423}, {"index": 3, "Person": "M. C. Begelman", "Fiability": 0.9905604124069214}, {"index": 4, "Person": "R. D. Blandford", "Fiability": 0.9953071475028992}, {"index": 5, "Person": "M. J. Rees", "Fiability": 0.9941856861114502}, {"index": 6, "Person": "L. Blecha", "Fiability": 0.9919770956039429}, {"index": 7, "Person": "T. J. Cox", "Fiability": 0.9935681819915771}, {"index": 8, "Person": "A. Loeb", "Fiability": 0.9811218976974487}, {"index": 9, "Person": "L. Hernquist", "Fiability": 0.9158643484115601}, {"index": 10, "Person": "P. R. Capelo", "Fiability": 0.9917762279510498}, {"index": 11, "Person": "M. Volonteri", "Fiability": 0.9941989779472351}, {"index": 12, "Person": "M. Dotti", "Fiability": 0.995175838470459}, {"index": 13, "Person": "J. M. Bellovary", "Fiability": 0.991267383

## You can go in the folder Download or Teacher to see the jsonfile